In [5]:
# Python Section
# Setups
import pandas as pd
import plotly.express as px


data_location = "datasets/"


baseball_players = pd.read_csv(data_location + "baseball_players.csv")
hitting_stats = pd.read_csv(data_location + "exit_velocity.csv")


#print(hitting_stats)
exp_stats = pd.read_csv(data_location + "expected_stats.csv")
exp_stats = exp_stats.rename(columns={'last_name':'Name',
                                      ' first_name':'to_drop',
                                      'player_id':'ID',
                                      'year':'Year',
                                      'pa':'Plate Appearances',
                                      'bip':'Balls In Play',
                                      'ba':'Batting Average',
                                      'est_ba':'Estimated Batting Average',
                                      'est_ba_minus_ba_diff':'xBA - BA Difference',
                                      'slg':'Slugging',
                                      'est_slg':'Estimated Slugging',
                                      'est_slg_minus_slg_diff':'Estimated Slugging Minus Slugging Difference',
                                      'woba':'Weighted On Base Average',
                                      'est_woba':'Estimated Weighted On Base Average',
                                      'est_woba_minus_woba_diff':'Estimated Weighted On Base Average Minus Weighted On Base Average Difference'})



hello awpirugbwaiuwav ape9uechbaepuvhbseviusbev

no u